## <strong> 1. 모델 구현 </strong>

2개의 합성곱 층(합성곱, 활성화 함수, 풀링)과 1개의 전결합층(Fully connected layer)로 구성된 모델을 만들어보자.

<h3> <strong> 1.1 도구 임포트, 입력 정의 </strong> </h3>

In [1]:
# 도구 임포트
import torch
import torch.nn as nn

임의의 텐서를 만들어 주자.

In [2]:
inputs = torch.Tensor(1, 1, 28, 28)

<h3> <strong> 1.2 합성곱층과 풀링 선언 </strong> </h3>

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)              # 입력 차원:1, 출력 차원: 32
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 입력 차원:32, 출력 차원: 64
pool = nn.MaxPool2d(2)                    

<h3> <strong> 1.3 구현체 연결 </strong> </h3>

위에서 만든 입력 텐서를 conv1에 넣어보자.

In [4]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


1차원 입력을 32개의 차원을 가진 텐서로 출력해주었다.

In [5]:
# fully connected Layer를 만들기 전까지 수행하여 out에 저장한다.
out = pool(out)
out = conv2(out)
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


[참고] .size(i)를 하면 i번째 차원의 크기를 알 수 있다.

In [6]:
print(out.size(0), out.size(1), out.size(2), out.size(3))

1 64 7 7


이제 해당 출력 텐서를 **.view()**를 사용해 펼쳐보자.

In [7]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합되었다. <br>
이제 출력층으로 10개의 뉴런을 배치해서 10개의 차원의 텐서로 변환하면 된다.

In [8]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## <strong> 2. CNN으로 MNIST 분류하기 </strong>

In [51]:
# import tools
import torch
import torch.nn
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# set device
device = 'cuda' if torch.cuda.is_available else 'cpu'

torch.manual_seed(777)  # fix random seed

if device == 'cuda':    # if use cuda fix random seed
  torch.cuda.manual_seed_all(777)

# set hyper-parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# define datasets
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True,         
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', 
                          train=False,         
                          transform=transforms.ToTensor(),
                          download=True)

data_loader = DataLoader(mnist_train, batch_size, shuffle=True, drop_last=True)

# design model
class CNN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # first layer(convolution)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # second layer(convolution)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # third layer(fully-connected layer)
    self.layer3 = torch.nn.Linear(7*7*64, 10, bias=True)
    
    # fully-connected layer's weight initialize
    torch.nn.init.xavier_uniform_(self.layer3.weight)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.layer3(out)
    return out
  
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)

model.train()
for epoch in range(training_epochs):
  avg_cost = 0
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    
    hypothesis = model(x)
    cost = criterion(hypothesis, y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.225604847
[Epoch:    2] cost = 0.0630015135
[Epoch:    3] cost = 0.0462957956
[Epoch:    4] cost = 0.037392702
[Epoch:    5] cost = 0.031350553
[Epoch:    6] cost = 0.0260586087
[Epoch:    7] cost = 0.0217555109
[Epoch:    8] cost = 0.0182944853
[Epoch:    9] cost = 0.016116567
[Epoch:   10] cost = 0.0133719705
[Epoch:   11] cost = 0.0103800893
[Epoch:   12] cost = 0.0103182914
[Epoch:   13] cost = 0.0077088601
[Epoch:   14] cost = 0.00677035563
[Epoch:   15] cost = 0.00714943279


In [52]:
# [참고용]
print(len(data_loader))              # DataLoader의 총 iteration
print(len(list(data_loader)[0]))     # feature matrix, label
print(list(data_loader)[0][0].shape) # bach size에 맞게 짝지어진 feature matrix

600
2
torch.Size([100, 1, 28, 28])


In [53]:
test_loader = DataLoader(mnist_test, batch_size=len(mnist_test), shuffle=False) # (10,000, 1, 28, 28) 크기로 바꾸기 위함

model.eval()

with torch.no_grad():
  for x, y in test_loader:
    x = x.to(device)
    y = y.to(device)
    prediction = model(x)
    correct_prediction = torch.argmax(prediction, 1) == y
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9884999990463257


출처: https://wikidocs.net/63565